<a href="https://colab.research.google.com/github/daia99/nerf_pl/blob/master/NeRF_and_Friends.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notes

This notebook is able to run COLMAP for image pose estimation, NeRF (Lightning implementation), NeX-MPI, and NeRF-SH/PlenOctree extraction. 

Forks of NeRF, and PlenOctree test out improvements to initial training stability implemented in: https://github.com/google/mipnerf

Forks also contain calls os.system('rsync -r...') to save checkpoints somewhere in Google Drive

# Sources

- https://gist.github.com/kwea123/f0e8f38ff2aa94495dbfe7ae9219f75c (COLMAP in Colab)
- https://gist.github.com/kwea123/a3c541a325e895ef79ecbc0d2e6d7221 (NeRF PyTorch Lightning implementation)
- https://github.com/nex-mpi/nex-code/ (NeX-MPI source and Colab notebook)
- https://github.com/sxyu/plenoctree (PlenOctree source)

# Misc Utilities

Check GPU in current instance

In [ ]:
!nvidia-smi

Use this in case debugging with remote access to Colab instance is desired (https://github.com/WassimBenzarti/colab-ssh)

In [ ]:
# Install colab_ssh on google colab
!pip install colab_ssh --upgrade

from colab_ssh import launch_ssh_cloudflared, init_git_cloudflared
launch_ssh_cloudflared(password="AnyPasswordHere")

# Calculate Pose from Images
Note: 12-30 images for forward-facing scenes, and 50-100 images for 360 scenes are recommended

First, mount Google Drive to Colab to load data into workspace

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Install prerequisites

In [ ]:
!sudo apt-get install \
    git \
    cmake \
    build-essential \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-regex-dev \
    libboost-system-dev \
    libboost-test-dev \
    libeigen3-dev \
    libsuitesparse-dev \
    libfreeimage-dev \
    libgoogle-glog-dev \
    libgflags-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev \
    libcgal-qt5-dev

Install Ceres-solver (takes 10-20 mins)

In [ ]:
%cd /content
!sudo apt-get install libatlas-base-dev libsuitesparse-dev
!git clone https://ceres-solver.googlesource.com/ceres-solver
%cd ceres-solver
!git checkout $(git describe --tags) # Checkout the latest release
%mkdir build
%cd build
!cmake .. -DBUILD_TESTING=OFF -DBUILD_EXAMPLES=OFF
!make
!sudo make install

Install COLMAP (10-20 mins)

In [ ]:
!git clone https://github.com/colmap/colmap
%cd colmap
!git checkout dev
%mkdir build
%cd build
!cmake ..
!make
!sudo make install
!CC=/usr/bin/gcc-6 CXX=/usr/bin/g++-6 cmake ..

Clone LLFF for utils

In [ ]:
%cd /content
!git clone https://github.com/Fyusion/LLFF

Run images to pose utils

In [ ]:
%cd /content/LLFF
# change the path below to your data folder (the folder containing the `images` folder)
!python imgs2poses.py "/content/drive/My Drive/path/to/folder"

# NeRF PL

Install prerequisites

In [ ]:
%cd /content
!git clone --recursive https://github.com/daia99/nerf_pl # fork of https://github.com/kwea123/nerf_pl

%cd /content/nerf_pl
!pip install -r requirements.txt

%cd /content/nerf_pl/torchsearchsorted
!pip install .

Train your NeRF scene

In [ ]:
%cd /content/nerf_pl

import os
# set training configurations here
os.environ['ROOT_DIR'] = "/content/drive/My Drive/colab/nerf/nerf_llff_data/fern"
                         # directory containing the data
os.environ['IMG_W'] = "504" # image width (do not set too large)
os.environ['IMG_H'] = "378" # image height (do not set too large)
os.environ['NUM_EPOCHS'] = "30" # number of epochs to train (depending on how many images there are,
                                # 20~30 might be enough)
os.environ['EXP'] = "fern" # name of the experience (arbitrary)
os.environ['CKPT_PATH'] = '/content/drive/My Drive/ckpts/epoch=10.ckpt' # if you have PyTorch ckpt file saved to continue training

# In case of limited memory, reduce batch_size, or use N_importance with 64
# Default for forward-facing scenes using NDC. If training on 360 scenes, add flags --spheric and --use_disp
!python train.py \
   --dataset_name llff \
   --root_dir "$ROOT_DIR" \
   --N_importance 128 --img_wh $IMG_W $IMG_H \
   --num_epochs $NUM_EPOCHS --batch_size 1024 \
   --optimizer adam --lr 5e-4 \
   --lr_scheduler cosine \
   --exp_name $EXP

Create rendering from your trained NeRF scene

In [ ]:
os.environ['SCENE'] = 'flower'
os.environ['CKPT_PATH'] = '/content/drive/My Drive/ckpts/epoch=10.ckpt'

# spheric_poses flag for evaluating 360 scenes
!python eval.py \
   --root_dir "$ROOT_DIR" \
   --dataset_name llff --scene_name $SCENE \
   --img_wh $IMG_W $IMG_H --N_importance 64 --ckpt_path $CKPT_PATH --spheric_poses

# NeX

It works well for forward-facing scenes

Install prerequisites

In [ ]:
%cd /content
!pip install lpips
!git clone https://github.com/nex-mpi/nex-code.git
!mkdir -p /content/nex-code/data
!mkdir -p /content/nex-code/runs

Copy dataset scene to cloned repo

In [ ]:
!cp -r "/content/drive/My Drive/..." "/content/nex-code/data"

Load Tensorboard

In [ ]:
%cd /content/nex-code
%load_ext tensorboard
%tensorboard --logdir runs

Train NeX scene

In [ ]:
%cd /content/nex-code

# tensorboard will update every 2 epochs (72 step)
# scene leads to local copy of scene input, model_dir is arbitrary name location for checkpointing
# decrease layers/sublayers in MPI if oom errors occur, or reduce ray batch size.
%time !python train.py -scene data/Shiny -model_dir Shiny -epochs 80 -layers 12 -sublayers 6 -ray 8000 -tb_toc 50 -num_workers 2 -llff_width 400 -http -web_width 4096

Play rendered video

In [ ]:
from IPython.display import HTML
from base64 import b64encode

# change <model_dir> to name in model_dir arg as above
video_path = "runs/video_output/<model_dir>/video.mp4"
mp4 = open(video_path, "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width=400 controls>
      <source src="{data_url}" type="video/mp4" controls playsinline autoplay muted loop>
</video>
""")

Upload real-time web renderer for viewing

In [ ]:
# upload file to nex-mpi backend 🦉
# please don't hack our server. thank you 🙏
import os, uuid
from IPython.display import HTML
import time

if not os.path.isfile("/usr/bin/rclone"):
  get_ipython().system_raw("curl -s https://rclone.org/install.sh | sudo bash")
file_path = str(uuid.uuid4())
get_ipython().system_raw("wget -O rclone.conf https://gist.githubusercontent.com/pureexe/8a0774c5e9994e3ead4fa1be555d43a5/raw/984a5e412b9ae2a182aabc8cfae7e69d64c9e3f3/rclone.conf")
# change <model_dir> to name in model_dir arg as above
!rclone --config=rclone.conf copy "runs/html/<model_dir>" "nex:$file_path" --contimeout 60s --timeout 300s --retries 3 --low-level-retries 10 --fast-list --max-backlog 400000 --transfers 20 --checkers 40 --buffer-size 64M --drive-chunk-size 64M --disable copy --drive-acknowledge-abuse --drive-keep-revision-forever --drive-stop-on-upload-limit --drive-stop-on-download-limit --drive-skip-shortcuts --drive-use-trash=false -q

time.sleep(10) # we have to wait backend to process files a bit
webgl_url = "https://nex-mpi.github.io/viewer/viewer.html?scene=https://nex.mpi.workers.dev/{}".format(file_path)
print("Real time demo: {}".format(webgl_url))
HTML(f"""
<iframe height=450 width=700 src="{webgl_url}" style="border:0px;padding-top:10px"></iframe>
""")

# PlenOctree

Install prerequisites

In [ ]:
%cd /content
# fork of https://github.com/sxyu/plenoctree, contains plenoctree/nerf_sh/config/llff.yaml that can be edited to pass flags to Python scripts
!git clone https://github.com/daia99/plenoctree.git 
%cd /content/plenoctree
!pip install -r requirements.txt
!pip install --upgrade jax jaxlib==0.1.65+cuda110 -f https://storage.googleapis.com/jax-releases/jax_releases.html
!pip install PyYAML==5.4.1

Load scene data to project

In [ ]:
!mkdir -p /content/plenoctree/data
!cp -r "/content/drive/My Drive/..." "/content/plenoctree/data"

Resize image to reduce memory usage

In [ ]:
import numpy as np
import os, imageio

# original from https://github.com/Fyusion/LLFF/blob/master/llff/poses/pose_utils.py
def _minify(basedir, factors=[], resolutions=[]):
    needtoload = False
    for r in factors:
        imgdir = os.path.join(basedir, 'images_{}'.format(r))
        if not os.path.exists(imgdir):
            needtoload = True
    for r in resolutions:
        imgdir = os.path.join(basedir, 'images_{}x{}'.format(r[1], r[0]))
        if not os.path.exists(imgdir):
            needtoload = True
    if not needtoload:
        return
    
    from shutil import copy
    from subprocess import check_output
    
    imgdir = os.path.join(basedir, 'images')
    imgs = [os.path.join(imgdir, f) for f in sorted(os.listdir(imgdir))]
    imgs = [f for f in imgs if any([f.endswith(ex) for ex in ['JPG', 'jpg', 'png', 'jpeg', 'PNG']])]
    imgdir_orig = imgdir
    
    wd = os.getcwd()

    for r in factors + resolutions:
        if isinstance(r, int):
            name = 'images_{}'.format(r)
            resizearg = '{}%'.format(100./r)
        else:
            name = 'images_{}x{}'.format(r[1], r[0])
            resizearg = '{}x{}'.format(r[1], r[0])
        imgdir = os.path.join(basedir, name)
        if os.path.exists(imgdir):
            continue
            
        print('Minifying', r, basedir)
        
        os.makedirs(imgdir)
        check_output('cp {}/* {}'.format(imgdir_orig, imgdir), shell=True)
        
        ext = imgs[0].split('.')[-1]
        args = ' '.join(['mogrify', '-resize', resizearg, '-format', 'png', '*.{}'.format(ext)])
        print(args)
        os.chdir(imgdir)
        check_output(args, shell=True)
        os.chdir(wd)
        
        if ext != 'png':
            check_output('rm {}/*.{}'.format(imgdir, ext), shell=True)
            print('Removed duplicates')
        print('Done')

In [ ]:
!apt-get update
!apt-get install imagemagick

Scale down images

In [ ]:
%cd /content
scene_dir = "/content/plenoctree/data/lego" # to directory of scene in project

scale_factors = [4, 6] # input list of dim factors for each desired set of shrunken images
_minify(scene_dir, factors = scale_factors) 

Train NeRF-SH

In [ ]:
%cd /content/plenoctree
%load_ext tensorboard
%tensorboard --logdir ckpt

In [ ]:
%cd /content/plenoctree
!export DATA_ROOT=./data/NeRF/nerf_synthetic/
!export CKPT_ROOT=./data/Plenoctree/checkpoints/syn_sh16/
!export SCENE=chair
!export CONFIG_FILE=nerf_sh/config/blender

# read: https://jax.readthedocs.io/en/latest/gpu_memory_allocation.html
os.environ['XLA_PYTHON_CLIENT_ALLOCATOR'] = 'default' # platform (lower GPU memory usage) or default
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'true'
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION'] = '.5' # reduce if crashes due to low RAM

# for llff spherical scenes, extraction after lindisp arg training is not supported, so this is set to false (leading to reduced initial training stability)
!python -m nerf_sh.train \
    --train_dir $CKPT_ROOT/$SCENE/ \
    --config $CONFIG_FILE \
    --data_dir $DATA_ROOT/$SCENE/

!python -m nerf_sh.eval \
    --chunk 4096 \
    --train_dir $CKPT_ROOT/$SCENE/ \
    --config $CONFIG_FILE \
    --data_dir $DATA_ROOT/$SCENE/

Convert NeRF-SH model to PlenOctree and optimize. Possible to evaluate and compress the PlenOctree

In [ ]:
%cd /content/plenoctree
import os

!export DATA_ROOT=./data/NeRF/nerf_synthetic/
!export CKPT_ROOT=./data/Plenoctree/checkpoints/syn_sh16
!export SCENE=chair
!export CONFIG_FILE=nerf_sh/config/blender

# in config file, you can add arguments (reduce radius and increase bbox_scale to ensure extracted PlenOctree can load on their volrend tool)
!python -m octree.extraction \
    --train_dir $CKPT_ROOT/$SCENE/ --is_jaxnerf_ckpt \
    --config $CONFIG_FILE \
    --data_dir $DATA_ROOT/$SCENE/ \
    --output $CKPT_ROOT/$SCENE/octrees/tree.npz

!python -m octree.optimization \
    --input $CKPT_ROOT/$SCENE/tree.npz \
    --config $CONFIG_FILE \
    --data_dir $DATA_ROOT/$SCENE/ \
    --output $CKPT_ROOT/$SCENE/octrees/tree_opt.npz

!python -m octree.evaluation \
    --input $CKPT_ROOT/$SCENE/octrees/tree_opt.npz \
    --config $CONFIG_FILE \
    --data_dir $DATA_ROOT/$SCENE/

# [Optional] Only used for in-browser viewing.
!python -m octree.compression \
    $CKPT_ROOT/$SCENE/octrees/tree_opt.npz \
    --out_dir $CKPT_ROOT/$SCENE/ \
    --overwrite